In [1]:
import requests
import pandas as pd
import sqlite3

In [2]:
def retry_requests(url, playloads,  headers):

    for i in range(3):
        try:
            return requests.get(url, params = payloads, headers = headers)
        except:
            print("發生錯誤,等待一分鐘後嘗試")
            time.sleep(60)
    return None
def get_daily_prices(date, connection):
    
    try:
        df = pd.read_sql("select * from daily_prices where 日期 = '{}'".format(date),
                         connection,
                         parse_dates=['日期'],
                         index_col=['證券代號', '日期'])
    except:
        df = pd.DataFrame()
        
    if not df.empty:
         return df, True
    url = "https://www.twse.com.tw/exchangeReport/MI_INDEX"
    
    payloads = {
    "response":"html",
    "date": date.strftime("&Y&m&d"),
    "type":"ALLBUT0999"
    }
    
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"
    }

    response = requests.get(url, params = payloads, headers = headers)

    try:
        df = pd.read_html(response.text)[-1]
    except:
        print('{} 找不到資料'.format(date.strftime('%Y%m%d')))
        return None, False
    
    df.columns =df.columns.get_level_values(2)
    
    df["漲跌價差"]=df["漲跌價差"].where(df['漲跌(+/-)']!="-", - df["漲跌價差"])
    
    df.drop(["證券名稱", "漲跌(+/-)"], inplace=True, axis=1)
    
    df["日期"] = pd.to_datetime(date)
    
    df = df.set_index(["證券代號", "日期"])
    
    df = df.apply(pd.to_numeric, errors="coerce")
    
    df.drop(df[df["收盤價"].isnull()].index, inplace=True)
    
    df['昨日收盤價'] = df['收盤價'] - df['漲跌價差']
    
    df['股價振幅'] = (df['最高價'] - df['最低價']) / df['昨日收盤價']*100
    
    return df, False 

In [5]:
connection = sqlite3.connect('date.ab')

df,in_db = get_daily_prices(pd.to_datetime('2021/7/1'),connection)

print(in_db)
df

False


,,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,昨日收盤價,股價振幅
證券代號,日期,,,,,,,,,,,,,,,
0050,2021-07-01,4680733,5327,649025587,139.0,139.20,138.05,138.30,-0.65,138.25,86,138.30,9,0.00,138.95,0.827636
0051,2021-07-01,175374,120,10152802,58.2,59.10,57.40,57.90,-0.30,57.40,6,58.20,7,0.00,58.20,2.920962
0052,2021-07-01,514042,270,64127738,125.0,125.20,124.35,124.35,-0.65,124.35,34,124.55,1,0.00,125.00,0.680000
0053,2021-07-01,12108,14,803303,66.7,66.70,66.20,66.20,-0.60,66.25,1,66.55,1,0.00,66.80,0.748503
0054,2021-07-01,4481,3,141086,31.9,31.90,31.56,31.56,-0.34,31.48,25,31.89,18,0.00,31.90,1.065831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,2021-07-01,298790,151,6575127,22.2,22.20,21.75,21.90,-0.25,21.85,4,21.90,5,29.59,22.15,2.031603
9945,2021-07-01,4105434,2624,233052391,56.6,57.40,56.30,56.60,0.00,56.60,71,56.70,21,6.94,56.60,1.943463
9946,2021-07-01,99493,64,1466349,14.7,14.75,14.70,14.70,0.00,14.70,33,14.75,23,91.88,14.70,0.340136


In [6]:
from trading_calendars import get_calendar

tw_calendar = get_calendar('XTAI')

tw_calendar.opens

date = pd.to_datetime('2021/6/11')

if date not in tw_calendar.opens:
    print(date.date(),'非交易日')
else:
    print(date.date(),'交易日')

2021-06-11 交易日


<ipython-input-6-8a761696dd19>:9: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  if date not in tw_calendar.opens:


In [7]:
import time
from trading_calendars import get_calendar

def update_daily_prices(start_date, end_date, connection):
    
    tw_calendar = get_calendar('XTAI')
    
    main_df = pd.DataFrame()
    
    for date in pd.date_range(start_date, end_date):
    
        if date not in tw_calendar.opens:
                continue
    
        df, in_db = get_daily_prices(date, connection)
    
        if df is not None and not in_db:
            main_df = main_df.append(df, sort=False)
            print('{} 每日收盤行情抓取完成，等待15秒'.format(date.strftime('%Y%m%d')))
            time.sleep(15)
    
    if not main_df.empty:
        main_df.to_sql('daily_prices', connection, if_exists='append')
        return main_df

In [8]:
connection = sqlite3.connect('data.db')

update_daily_prices('2021/6/29', '2021/7/1', connection)

<ipython-input-7-ab117e90e502>:12: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  if date not in tw_calendar.opens:


20210629 每日收盤行情抓取完成，等待15秒


<ipython-input-7-ab117e90e502>:12: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  if date not in tw_calendar.opens:


20210630 每日收盤行情抓取完成，等待15秒


<ipython-input-7-ab117e90e502>:12: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  if date not in tw_calendar.opens:


20210701 每日收盤行情抓取完成，等待15秒


,,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,昨日收盤價,股價振幅
證券代號,日期,,,,,,,,,,,,,,,
0050,2021-06-29,4680733,5327,649025587,139.0,139.20,138.05,138.30,-0.65,138.25,86,138.30,9,0.00,138.95,0.827636
0051,2021-06-29,175374,120,10152802,58.2,59.10,57.40,57.90,-0.30,57.40,6,58.20,7,0.00,58.20,2.920962
0052,2021-06-29,514042,270,64127738,125.0,125.20,124.35,124.35,-0.65,124.35,34,124.55,1,0.00,125.00,0.680000
0053,2021-06-29,12108,14,803303,66.7,66.70,66.20,66.20,-0.60,66.25,1,66.55,1,0.00,66.80,0.748503
0054,2021-06-29,4481,3,141086,31.9,31.90,31.56,31.56,-0.34,31.48,25,31.89,18,0.00,31.90,1.065831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,2021-07-01,298790,151,6575127,22.2,22.20,21.75,21.90,-0.25,21.85,4,21.90,5,29.59,22.15,2.031603
9945,2021-07-01,4105434,2624,233052391,56.6,57.40,56.30,56.60,0.00,56.60,71,56.70,21,6.94,56.60,1.943463
9946,2021-07-01,99493,64,1466349,14.7,14.75,14.70,14.70,0.00,14.70,33,14.75,23,91.88,14.70,0.340136
